In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,63,2024-03-08,2,116.612903,97.951613,0.558677,11.425806,25.708065,0.207710,118.911290,...,9.962903,24.917742,0.236565,119.259677,0,3,4,3,19,2024
1,63,2024-03-08,-5,107.290323,97.087097,0.527323,11.975806,21.546774,0.167758,110.217742,...,11.740323,19.962903,0.172339,111.404839,0,3,4,3,5,2024
2,64,2024-03-08,5,111.444444,96.611111,0.544413,12.742857,25.939683,0.224381,114.709524,...,11.016129,29.545161,0.199903,118.770968,0,3,4,2,22,2024
3,63,2024-03-08,1,113.338710,96.620968,0.562710,12.754839,23.116129,0.222952,116.666129,...,11.541935,23.993548,0.182726,116.840323,0,3,4,4,18,2024
4,62,2024-03-08,2,110.754098,96.018033,0.538377,11.263934,22.183607,0.222230,115.178689,...,10.485484,20.677419,0.207113,121.259677,0,3,4,3,16,2024
5,63,2024-03-08,2,120.225806,100.232258,0.538919,10.832258,27.656452,0.216032,119.090323,...,12.784127,22.395238,0.184746,108.252381,0,3,4,2,1,2024
6,64,2024-03-08,-1,120.857143,100.233333,0.571587,10.971429,21.668254,0.221016,119.976190,...,11.837500,19.485937,0.211297,116.040625,0,3,4,2,17,2024
7,63,2024-03-08,-1,112.854839,98.516129,0.524323,11.003226,24.825806,0.201726,113.693548,...,12.562295,26.695082,0.194754,110.259016,0,3,4,2,11,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-08,NOP,PHI,1.327869,3.450000,0
1,2024-03-08,CHO,WAS,2.100000,1.769231,0
2,2024-03-08,ORL,NYK,1.847458,1.980392,0
3,2024-03-08,MIN,CLE,1.952381,1.869565,0
4,2024-03-08,MIA,OKC,3.850000,1.277778,0
5,2024-03-08,ATL,MEM,1.704225,2.200000,1
6,2024-03-08,MIL,LAL,1.769231,2.100000,0
7,2024-03-08,HOU,POR,1.444444,2.850000,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-08,NOP,PHI,1.327869,3.450000,0
1,2024-03-08,CHO,WAS,2.100000,1.769231,0
2,2024-03-08,ORL,NYK,1.847458,1.980392,0
3,2024-03-08,MIN,CLE,1.952381,1.869565,0
4,2024-03-08,MIA,OKC,3.850000,1.277778,0
5,2024-03-08,ATL,MEM,1.704225,2.200000,1
6,2024-03-08,MIL,LAL,1.769231,2.100000,0
7,2024-03-08,HOU,POR,1.444444,2.850000,1


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-08,NOP,PHI,1.327869,3.450000,0
1,2024-03-08,CHO,WAS,2.100000,1.769231,0
2,2024-03-08,ORL,NYK,1.847458,1.980392,0
3,2024-03-08,MIN,CLE,1.952381,1.869565,0
4,2024-03-08,MIA,OKC,3.850000,1.277778,0
5,2024-03-08,ATL,MEM,1.704225,2.200000,1
6,2024-03-08,MIL,LAL,1.769231,2.100000,0
7,2024-03-08,HOU,POR,1.444444,2.850000,1


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-08,NOP,PHI,1.327869,3.450000,0
1,2024-03-08,CHO,WAS,2.100000,1.769231,0
2,2024-03-08,ORL,NYK,1.847458,1.980392,0
3,2024-03-08,MIN,CLE,1.952381,1.869565,0
4,2024-03-08,MIA,OKC,3.850000,1.277778,0
5,2024-03-08,ATL,MEM,1.704225,2.200000,1
6,2024-03-08,MIL,LAL,1.769231,2.100000,0
7,2024-03-08,HOU,POR,1.444444,2.850000,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-08,NOP,PHI,1.327869,3.450000,1.0
1,2024-03-08,CHO,WAS,2.100000,1.769231,0.0
2,2024-03-08,ORL,NYK,1.847458,1.980392,1.0
3,2024-03-08,MIN,CLE,1.952381,1.869565,1.0
4,2024-03-08,MIA,OKC,3.850000,1.277778,0.0
5,2024-03-08,ATL,MEM,1.704225,2.200000,1.0
6,2024-03-08,MIL,LAL,1.769231,2.100000,1.0
7,2024-03-08,HOU,POR,1.444444,2.850000,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-08,NOP,PHI,1.327869,3.450000,1.0
1,2024-03-08,CHO,WAS,2.100000,1.769231,0.0
2,2024-03-08,ORL,NYK,1.847458,1.980392,1.0
3,2024-03-08,MIN,CLE,1.952381,1.869565,1.0
4,2024-03-08,MIA,OKC,3.850000,1.277778,0.0
5,2024-03-08,ATL,MEM,1.704225,2.200000,1.0
6,2024-03-08,MIL,LAL,1.769231,2.100000,1.0
7,2024-03-08,HOU,POR,1.444444,2.850000,1.0
